In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [2]:
bnf_pandas = pd.read_csv("../data/indices/NIFTY BANK.csv")

In [3]:
# symbol = 'midcp'
# symbol = 'nifty'
# symbol = 'fnf'
symbol = 'bnf'
# symbol = 'sensex'
# symbol = 'bankex'

if symbol == 'bnf' or symbol == 'bankex':
    LEVERAGE_ = 5
    LOT_SIZE_ = 15
    SLIPPAGE_ = 0.0001
elif symbol == 'nifty' or symbol == 'fnf':
    LEVERAGE_ = 7
    LOT_SIZE_ = 25
    SLIPPAGE_ = 0.0002
elif symbol == 'midcp':
    LEVERAGE_ = 8
    LOT_SIZE_ = 50
    SLIPPAGE_ = 0.0005
elif symbol == 'sensex':
    LEVERAGE_ = 8
    LOT_SIZE_ = 10
    SLIPPAGE_ = 0.0001

PORTFOLIO = 1000000
print(LEVERAGE_)

5


In [4]:
bnf_pandas.head()

,datetime,o,h,l,c
0,2017-01-02T09:15:00.000000,18242.3000,18248.2000,18175.9000,18181.2000
1,2017-01-02T09:16:00.000000,18181.8500,18194.7000,18179.9500,18184.4500
2,2017-01-02T09:17:00.000000,18184.9500,18189.2500,18133.8000,18133.8000
3,2017-01-02T09:18:00.000000,18135.1000,18141.5500,18118.5500,18138.9500
4,2017-01-02T09:19:00.000000,18138.9500,18142.5500,18120.4500,18124.3000


In [5]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["datetime"], inplace=True)
# bnf_pandas

In [6]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [7]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

In [8]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )

In [9]:
bnf2 = resample(bnf, '20m', '15m')
bnf_pandas = bnf2.to_pandas()
bnf_pandas.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'}, inplace=True)

In [10]:
def ma_crossover_logic(df, fast_ma, slow_ma):
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    
    fast_ema = df['close'].ewm(span=fast_ma, adjust=False).mean()
    slow_ema = df['close'].ewm(span=slow_ma, adjust=False).mean()
    
    df['fast_ma'] = fast_ema
    df['slow_ma'] = slow_ema
    
    df['signal'] = 0  # Default to no signal
    df.loc[fast_ema > slow_ema, 'signal'] = 1   # Long Signal
    df.loc[fast_ema < slow_ema, 'signal'] = -1  # Short Signal
    
    return df

In [15]:
import pandas as pd

def execute(df, portfolio=10000000, leverage=6, lot_size=1, slippage=0.0001, rpt=2):
    trade_book = []
    in_trade = None  # Track current position ('LONG' or 'SHORT')
    entry_price = 0
    entry_time = None

    for i in range(1, len(df)):
        current_signal = df.iloc[i]['signal']
        current_close = df.iloc[i]['close']
        current_time = df.iloc[i]['datetime']
        points = 0

        if in_trade:
            if (in_trade == 'LONG' and current_signal == -1) or (in_trade == 'SHORT' and current_signal == 1):
                # Exit current position and reverse trade
                exit_price = current_close
                exit_time = current_time
                points = exit_price - entry_price if in_trade == 'LONG' else entry_price - exit_price
                
                # qty = int(round((portfolio * rpt / 100) / abs(points)) / lot_size) * lot_size
                qty = int(round(portfolio * leverage / entry_price)/lot_size) * lot_size
                slippage_ = slippage * (entry_price + exit_price)
                final_points = points - slippage_
                pnl = final_points * qty
                roi = (pnl / portfolio) * 100
                
                trade_book.append({
                    "Trade Type": in_trade,
                    "Entry Time": entry_time,
                    "Entry Price": entry_price,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Points Captured": points,
                    "Slippage": slippage_,
                    "Qty": qty,
                    "Final Points": final_points,
                    "PnL": pnl,
                    "ROI%": roi,
                    "Trade Duration": exit_time - entry_time,
                    "Remarks": "Signal Reversal",
                })
                
                # Reverse trade
                in_trade = 'LONG' if current_signal == 1 else 'SHORT'
                entry_price = current_close
                entry_time = current_time
        
        elif current_signal in [1, -1]:
            # New trade entry
            in_trade = 'LONG' if current_signal == 1 else 'SHORT'
            entry_price = current_close
            entry_time = current_time
    
    return pd.DataFrame(trade_book)


In [16]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [17]:
PORTFOLIO = 50_00_000
LEVERAGE_ = 5
LOT_SIZE_ = 30
SLIPPAGE_ = 0.0001
RPT_ = 2

In [18]:
stats_dictionary = {}

slow_ma = 20
fast_ma = 10
# trailing_ma = 80
sl_pct = 1

df = ma_crossover_logic(bnf_pandas, fast_ma, slow_ma)

tb = execute(df)
variation = f'MA1 : {slow_ma}, MA2 : {fast_ma}, SL : {sl_pct}%'

In [19]:
tb['Trade Year'] = tb['Entry Time'].dt.year
tb = tb.sort_values(by="Entry Time")
stats = generate_stats(tb, variation)

In [20]:
for overall_roi_dd_ratio, stats_df in stats.items():
    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
        print(stats_df.to_string())
        stats_dictionary[overall_roi_dd_ratio] = stats_df

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                    Variation
2017     108.0456          170  32.3529                6.6895             -2.2598     -57.0129       1.8951  MA1 : 20, MA2 : 10, SL : 1%
2018     141.5200          186  33.8710                7.2213             -2.5482     -51.1023       2.7693  MA1 : 20, MA2 : 10, SL : 1%
2019     126.4402          175  33.1429                9.1868             -3.4734     -55.9326       2.2606  MA1 : 20, MA2 : 10, SL : 1%
2020     280.9319          179  34.6369               18.7571             -7.5385    -132.5475       2.1195  MA1 : 20, MA2 : 10, SL : 1%
2021     178.5916          182  34.0659               10.8257             -4.1050     -91.2694       1.9568  MA1 : 20, MA2 : 10, SL : 1%
2022      37.1244          182  29.6703               10.5835             -4.1749     -98.3531       0.3775  MA1 : 20, MA2 : 10, SL : 1%
2023      96.8563          163  31.2883  

In [23]:
tb.tail(250)

,Trade Type,Entry Time,Entry Price,Exit Time,Exit Price,Points Captured,Slippage,Qty,Final Points,PnL,ROI%,Trade Duration,Remarks,Trade Year
1194,SHORT,2023-09-05 13:15:00,44433.3500,2023-09-05 15:15:00,44543.8500,-110.5000,8.8977,1350,-119.3977,-161186.9220,-1.6119,0 days 02:00:00,Signal Reversal,2023
1195,LONG,2023-09-05 15:15:00,44543.8500,2023-09-06 09:55:00,44469.3500,-74.5000,8.9013,1347,-83.4013,-112341.5780,-1.1234,0 days 18:40:00,Signal Reversal,2023
1196,SHORT,2023-09-06 09:55:00,44469.3500,2023-09-07 10:15:00,44462.8500,6.5000,8.8932,1349,-2.3932,-3228.4538,-0.0323,1 days 00:20:00,Signal Reversal,2023
1197,LONG,2023-09-07 10:15:00,44462.8500,2023-09-13 09:15:00,45355.1000,892.2500,8.9818,1349,883.2682,1191528.8085,11.9153,5 days 23:00:00,Signal Reversal,2023
1198,SHORT,2023-09-13 09:15:00,45355.1000,2023-09-13 10:55:00,45610.2500,-255.1500,9.0965,1323,-264.2465,-349598.1658,-3.4960,0 days 01:40:00,Signal Reversal,2023
1199,LONG,2023-09-13 10:55:00,45610.2500,2023-09-18 12:55:00,46111.9500,501.7000,9.1722,1315,492.5278,647674.0307,6.4767,5 days 02:00:00,Signal Reversal,2023
1200,SHORT,2023-09-18 12:55:00,46111.9500,2023-09-25 12:55:00,44859.9500,1252.0000,9.0972,1301,1242.9028,1617016.5558,16.1702,7 days 00:00:00,Signal Reversal,2023
1201,LONG,2023-09-25 12:55:00,44859.9500,2023-09-26 10:15:00,44650.0500,-209.9000,8.9510,1337,-218.8510,-292603.7870,-2.9260,0 days 21:20:00,Signal Reversal,2023
1202,SHORT,2023-09-26 10:15:00,44650.0500,2023-09-27 13:55:00,44591.1500,58.9000,8.9241,1344,49.9759,67167.5827,0.6717,1 days 03:40:00,Signal Reversal,2023
1203,LONG,2023-09-27 13:55:00,44591.1500,2023-09-28 11:35:00,44512.5500,-78.6000,8.9104,1346,-87.5104,-117788.9580,-1.1779,0 days 21:40:00,Signal Reversal,2023


In [24]:
tb['Trade Duration'].mean()

Timedelta('2 days 01:09:54.182825484')

In [23]:
# tb.to_csv('full_ma_cross_4_20_MAs_15m_TF.csv')

In [94]:
sl_pct_range = [0.5, 0.75, 1, 1.25, 1.5]
stats_dictionary = {}

PORTFOLIO = 1_00_00_000
LEVERAGE_ = 5
LOT_SIZE_ = 30
SLIPPAGE_ = 0.0001

for i in range(4, 41, 2):
    for j in range(6, 81, 2):
        for sl in sl_pct_range:
            if i < j and ((j-i) <= 16):
                variation = f'MA1 : {i}, MA2 : {j}, SL : {sl}%'
                print(variation)
                df = ma_crossover_logic(bnf_pandas, i, j)
                tb = execute(df, sl, 1, PORTFOLIO, LEVERAGE_, LOT_SIZE_, SLIPPAGE_, 3)
                if len(tb) > 0:
                    tb['Trade Year'] = tb['Entry Time'].dt.year
                    tb = tb.sort_values(by="Entry Time")
                    stats = generate_stats(tb, variation)

                    for overall_roi_dd_ratio, stats_df in stats.items():
                        if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 18:
                            print(stats_df.to_string())
                            stats_dictionary[overall_roi_dd_ratio] = stats_df

MA1 : 4, MA2 : 6, SL : 0.5%
MA1 : 4, MA2 : 6, SL : 0.75%
MA1 : 4, MA2 : 6, SL : 1%
MA1 : 4, MA2 : 6, SL : 1.25%
MA1 : 4, MA2 : 6, SL : 1.5%
MA1 : 4, MA2 : 8, SL : 0.5%
MA1 : 4, MA2 : 8, SL : 0.75%
MA1 : 4, MA2 : 8, SL : 1%
MA1 : 4, MA2 : 8, SL : 1.25%
MA1 : 4, MA2 : 8, SL : 1.5%
MA1 : 4, MA2 : 10, SL : 0.5%
MA1 : 4, MA2 : 10, SL : 0.75%
MA1 : 4, MA2 : 10, SL : 1%
MA1 : 4, MA2 : 10, SL : 1.25%
MA1 : 4, MA2 : 10, SL : 1.5%
MA1 : 4, MA2 : 12, SL : 0.5%
MA1 : 4, MA2 : 12, SL : 0.75%
MA1 : 4, MA2 : 12, SL : 1%
MA1 : 4, MA2 : 12, SL : 1.25%
MA1 : 4, MA2 : 12, SL : 1.5%
MA1 : 4, MA2 : 14, SL : 0.5%
MA1 : 4, MA2 : 14, SL : 0.75%
MA1 : 4, MA2 : 14, SL : 1%
MA1 : 4, MA2 : 14, SL : 1.25%
MA1 : 4, MA2 : 14, SL : 1.5%
MA1 : 4, MA2 : 16, SL : 0.5%
MA1 : 4, MA2 : 16, SL : 0.75%
MA1 : 4, MA2 : 16, SL : 1%
MA1 : 4, MA2 : 16, SL : 1.25%
MA1 : 4, MA2 : 16, SL : 1.5%
MA1 : 4, MA2 : 18, SL : 0.5%
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio    

KeyboardInterrupt: 